## DETERMINING HABITABLE ZONES ##

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from astropy import constants as const
from astropy import units as u

# DEFYNING CONSTANTS

In [3]:
# Definiendo constantes
G = const.G.to(u.cm**3/(u.g*u.s**2)) # Constante Gravitacional en cm^3/(g*s^2)
Msol = const.M_sun.to(u.g) # Masa solar en gramos
R_sol = const.R_sun.to(u.cm)  # Radio del sol en centímetros

In [4]:
# Read the CSV file with space as the delimiter
exoplanet_data = pd.read_csv('exoplanet_data.csv', sep=',', comment='#')
exoplanet_data

,Planet_name,Host_name,gaia_id,Number_of_stars,Number_of_planets,Number_of_moons,disc_facility,Planet_controversial_flag,Orbital_period[days],delta_orbpererr1,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,st_nspec
0,K2-111 b,K2-111,Gaia DR2 53006669599267328,1,2,0,K2,0,5.351800,4.000000e-04,...,11.130,0.034,-0.034,9.377000,0.021,-0.021,10.92940,0.000621,-0.000621,0
1,TOI-1695 b,TOI-1695,Gaia DR2 534988616816537728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.134279,7.100000e-06,...,12.989,0.081,-0.081,8.818000,0.021,-0.021,12.13640,0.000460,-0.000460,0
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,1.841000e-03,...,12.668,0.114,-0.114,11.075000,0.021,-0.021,12.49590,0.000222,-0.000222,0
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306,0
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,2.021000e-05,...,13.752,0.103,-0.103,12.355000,0.021,-0.021,13.66050,0.000228,-0.000228,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Kepler-99 b,Kepler-99,Gaia DR2 2076871091425583232,2,1,0,Kepler,0,4.603580,NaN,...,13.206,0.092,-0.092,10.757000,0.018,-0.018,12.96180,0.000373,-0.000373,0
701,HIP 29442 d,HD 42813,Gaia DR2 2993561629444856960,1,3,0,Paranal Observatory,0,6.429575,2.600000e-05,...,9.490,0.030,-0.030,7.587000,0.024,-0.024,9.27468,0.000294,-0.000294,0
702,TOI-1467 b,TOI-1467,Gaia DR2 403141126645929728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,5.971147,8.746000e-06,...,12.293,0.017,-0.017,8.568000,0.019,-0.019,11.56220,0.000875,-0.000875,0
703,EPIC 201595106 b,EPIC 201595106,Gaia DR2 3701123406596146048,1,2,0,K2,0,0.877240,4.000000e-05,...,11.746,0.069,-0.069,10.218000,0.021,-0.021,11.64770,0.000245,-0.000245,0


In [5]:
main_stars = exoplanet_data[exoplanet_data['Number_of_stars']== 1]
main_stars


,Planet_name,Host_name,gaia_id,Number_of_stars,Number_of_planets,Number_of_moons,disc_facility,Planet_controversial_flag,Orbital_period[days],delta_orbpererr1,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,st_nspec
0,K2-111 b,K2-111,Gaia DR2 53006669599267328,1,2,0,K2,0,5.351800,4.000000e-04,...,11.130,0.034,-0.034,9.377000,0.021,-0.021,10.92940,0.000621,-0.000621,0
1,TOI-1695 b,TOI-1695,Gaia DR2 534988616816537728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.134279,7.100000e-06,...,12.989,0.081,-0.081,8.818000,0.021,-0.021,12.13640,0.000460,-0.000460,0
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,1.841000e-03,...,12.668,0.114,-0.114,11.075000,0.021,-0.021,12.49590,0.000222,-0.000222,0
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306,0
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,2.021000e-05,...,13.752,0.103,-0.103,12.355000,0.021,-0.021,13.66050,0.000228,-0.000228,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Kepler-102 d,Kepler-102,Gaia DR2 2119583201145735808,1,5,0,Kepler,0,10.311767,4.100000e-06,...,12.072,0.027,-0.027,9.351000,0.011,-0.011,11.47840,0.000248,-0.000248,0
701,HIP 29442 d,HD 42813,Gaia DR2 2993561629444856960,1,3,0,Paranal Observatory,0,6.429575,2.600000e-05,...,9.490,0.030,-0.030,7.587000,0.024,-0.024,9.27468,0.000294,-0.000294,0
702,TOI-1467 b,TOI-1467,Gaia DR2 403141126645929728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,5.971147,8.746000e-06,...,12.293,0.017,-0.017,8.568000,0.019,-0.019,11.56220,0.000875,-0.000875,0
703,EPIC 201595106 b,EPIC 201595106,Gaia DR2 3701123406596146048,1,2,0,K2,0,0.877240,4.000000e-05,...,11.746,0.069,-0.069,10.218000,0.021,-0.021,11.64770,0.000245,-0.000245,0


## Calculando zonas de habitabilidad 

Basándonos en el modelo de  (Kopparapu,2013), el cual considera planetas rocosos aquellos que tienen una masa menor a $5M_{\oplus}$. 
Este modelo es 

hola